In [1]:
import cv2
from scipy import ndimage
import matplotlib.pyplot as plt
import os
import numpy as np
import random
import itertools

In [2]:
def Meanimage(img1,img2):
    img2 = cv2.resize(img2,(img1.shape[1],img1.shape[0]),interpolation=cv2.INTER_CUBIC)
    mean_img = np.mean([img1, img2], axis=0)
    img = np.asarray(np.uint8(mean_img))
    return img
        

In [3]:
class data_augmentation():
    
    def __init__(self, PATH = "../LICHENI", excluded_folder = []):
        self.excluded_folder = excluded_folder
        self.PATH = PATH
        self.lista_cartelle = os.listdir(PATH)
        self.train_dic, self.traindata, self.trainlabel = self.creodict()

    
    
    
    
    
    def creodict(self):
        # creo dict con chiavi i path per le immagini e valori le classi
        train = []
        train_images = []
        label = []
        for cartelle in self.lista_cartelle:
            if(cartelle in self.excluded_folder):
                continue           
            specie = os.listdir(self.PATH + "/" + cartelle)
            for spec in specie:
                immagini = os.listdir(self.PATH + "/" + cartelle + "/" + spec)
                for images in immagini:
                    train.append(self.PATH + "/" + cartelle + "/" + spec + "/" + images)
                    label.append(spec)
        for tr in train:
            train_images.append(cv2.imread(tr))         
        return dict(zip(train, label)), train_images, label
    
    
    def SamplePairing(self, number_of_new_images = 10,all_pox = False):
        training_species = list(self.train_dic.keys())
        training_labels = list(self.train_dic.values())
        if(all_pox == False):
            cont = 0
            coppie_create = []
            while(cont < number_of_new_images):
                ran1 = random.randint(0,len(training_species) - 1)
                ran2 = random.randint(0,len(training_species) - 1)               
                if(ran1==ran2 or str(ran1) + " - " + str(ran2) in coppie_create):
                    continue
                else:
                    img1 = cv2.imread(training_species[ran1])
                    img2 = cv2.imread(training_species[ran2])
                    img = Meanimage(img1,img2)
                    self.traindata.append(img)
                    self.trainlabel.append(training_labels[ran1])
                    cont = cont + 1
                    coppie_create.append(str(ran1) + " - " + str(ran2))
        else:
            j = 0
            for i in  itertools.product(*[training_species,training_species]):
                j = j+1
                print(j)
                if(i[0]==i[1]):
                    continue
                else:                   
                    img1 = cv2.imread(i[0])
                    img2 = cv2.imread(i[1])
                    img = Meanimage(img1,img2)
                    self.traindata.append(img)
                    self.trainlabel.append(self.train_dic[i[0]])
                    
    def flipping(self):
        training_species = list(self.train_dic.keys())
        training_labels = list(self.train_dic.values())
        for ii,image in enumerate(training_species):
            img = cv2.imread(image)
            flipHorizontal = cv2.flip(img, 1)
            self.traindata.append(flipHorizontal)
            self.trainlabel.append(trainig_labels[ii])
                    
                    
            
            
        
    
    def rotation_augmentation(self, rot = [90,180,270]):
        training_species = list(self.train_dic.keys())
        training_labels = list(self.train_dic.values())
        for ii,images in enumerate(training_species):
            img = cv2.imread(images)
            for i in rot:
                rotated = ndimage.rotate(img, 90)
                self.traindata.append(rotated)
                self.trainlabel.append(training_labels[ii])
        
    
    def random_cropping(self, img, zone = "left"):
        """
        zone = {"random, left, right"}
        """
        temp = None
        if(zone == "left"):
            dim_y = random.randint(img.shape[0]//4,img.shape[0])
            dim_x = random.randint(img.shape[1]//4,img.shape[1])
            temp = img[0:dim_y,0:dim_x,:]
            temp = cv2.resize(temp,(img.shape[1],img.shape[0]),interpolation=cv2.INTER_CUBIC)
        elif(zone == "right"):
            dim_y = random.randint(img.shape[0]//4,3*img.shape[0]//4)
            dim_x = random.randint(img.shape[1]//4,3*img.shape[1]//4)
            temp = img[dim_y:img.shape[0],dim_x:img.shape[1],:]
            temp = cv2.resize(temp,(img.shape[1],img.shape[0]),interpolation=cv2.INTER_CUBIC)
        elif(zone=="random"):       
            start_y = random.randint(0, img.shape[0])
            dim_y = random.randint(0,img.shape[0])
            start_x = random.randint(0,img.shape[1])
            dim_x = random.randint(0,img.shape[1])
            temp = img[start_y:min(start_y+dim_y,img.shape[0] - 1),start_x:min(start_x+dim_x,img.shape[1]-1),:]
            temp = cv2.resize(temp,(img.shape[1],img.shape[0]),interpolation=cv2.INTER_CUBIC)
        else:
            raise Exception("Input is not valid")
        return temp
    
    def cropping(self, number_of_images = 100):
        training_species = list(self.train_dic.keys())
        training_labels = list(self.train_dic.values())
        cont = 0
        coppie_create = []
        zone = ["left","right","random"]
        while(cont < number_of_images):
            ran1 = random.randint(0,len(training_species) - 1)
            ran2 = random.randint(0,2)
            img1 = cv2.imread(training_species[ran1])
            self.traindata.append(self.random_cropping(img1,zone = zone[ran2]))
            self.trainlabel.append(training_labels[ran1])
            cont = cont + 1
            
            
            
            
        
        
    
    

In [4]:
da = data_augmentation(excluded_folder = ["super"])    

In [5]:
temp = da.rotation_augmentation()

In [7]:
da.cropping(number_of_images = 200)

1068